In [23]:
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET

In [24]:
#model = YOLO('yolov8n.pt')
#model.predict(
#   source='https://media.roboflow.com/notebooks/examples/dog.jpeg',
#   conf=0.25
#)

In [25]:
# !yolo predict model=yolov8n.pt source='/Users/erik/repos/ComputerVisionProject/img.png'

In [34]:
model = YOLO("yolov8n.yaml")

# Train the model
model.train(data="../nemoV1.yaml", epochs=1)

New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.41 🚀 Python-3.10.6 torch-2.2.2 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=../nemoV1.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

train: Scanning /Users/erik/repos/ComputerVisionProject/data/labels/train/2024_01_28_18_07_53... 109 images, 244 backgrounds, 0 corrupt: 100%|██████████| 353/353 [00:03<00:00, 110.05it/s]

train: New cache created: /Users/erik/repos/ComputerVisionProject/data/labels/train/2024_01_28_18_07_53.cache



val: Scanning /Users/erik/repos/ComputerVisionProject/data/labels/val/2024_02_01_18_41_23... 131 images, 164 backgrounds, 0 corrupt: 100%|██████████| 295/295 [00:03<00:00, 91.32it/s]

val: New cache created: /Users/erik/repos/ComputerVisionProject/data/labels/val/2024_02_01_18_41_23.cache


...


1 epochs completed in 0.046 hours.
Optimizer stripped from /Users/erik/repos/ComputerVisionProject/runs/detect/train16/weights/last.pt, 6.2MB
Optimizer stripped from /Users/erik/repos/ComputerVisionProject/runs/detect/train16/weights/best.pt, 6.2MB

Validating /Users/erik/repos/ComputerVisionProject/runs/detect/train16/weights/best.pt...
Ultralytics YOLOv8.1.41 🚀 Python-3.10.6 torch-2.2.2 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:28<00:00,  2.90s/it]

                   all        295        254          0          0          0          0


Speed: 1.5ms preprocess, 51.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /Users/erik/repos/ComputerVisionProject/runs/detect/train16


AttributeError: 'DetMetrics' object has no attribute 'curves_results'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass